Importing the necessary libraries

In [ ]:
import time
import cv2
from cv2  import dnn_superres
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim

Defined the input and output images directory path

In [ ]:
image_path='/content/images'
output_path='/content/outputs'

Intialized all the Image upscaling models for different scaling factors

In [ ]:
MODEL_LIST = [
    {"name": "EDSR", "path":"/content/drive/MyDrive/Image_Upscaling/EDSR_x2.pb", "scale":2},
    {"name": "ESPCN", "path":"/content/drive/MyDrive/Image_Upscaling/ESPCN_x2.pb", "scale":2},
    {"name": "FSRCNN", "path":"/content/drive/MyDrive/Image_Upscaling/FSRCNN_x2.pb", "scale":2},
    {"name": "LAPSRN", "path":"/content/drive/MyDrive/Image_Upscaling/LapSRN_x2.pb", "scale":2},

    {"name": "EDSR", "path":"/content/drive/MyDrive/Image_Upscaling/EDSR_x3.pb", "scale":3},
    {"name": "ESPCN", "path":"/content/drive/MyDrive/Image_Upscaling/ESPCN_x3.pb", "scale":3},
    {"name": "FSRCNN", "path":"/content/drive/MyDrive/Image_Upscaling/FSRCNN_x3.pb", "scale":3},

    {"name": "EDSR", "path":"/content/drive/MyDrive/Image_Upscaling/EDSR_x4.pb", "scale":4},
    {"name": "ESPCN", "path":"/content/drive/MyDrive/Image_Upscaling/ESPCN_x4.pb", "scale":4},
    {"name": "FSRCNN", "path":"/content/drive/MyDrive/Image_Upscaling/FSRCNN_x4.pb", "scale":4},
    {"name": "LAPSRN", "path":"/content/drive/MyDrive/Image_Upscaling/LapSRN_x4.pb", "scale":4},

    {"name": "LAPSRN", "path":"/content/drive/MyDrive/Image_Upscaling/LapSRN_x8.pb", "scale":8},
]

Functions to Calculate PSNR, MSE, SSIM score, and saving the output image in the above mentioned directory

In [ ]:
def calculate_psnr(original,new):
    return cv2.PSNR(original,new)

def calculate_mse(original,new):
    return (np.square(original - new)).mean(axis=None)

def calculate_ssim(original,new):
    return ssim(original, new, channel_axis=2)

def save_image(path, result):
    directory = os.path.dirname(path)
    if not os.path.exists(directory):
        os.makedirs(directory)
    cv2.imwrite(path, result)

This function, first downscale the image and the upscale it using the Models define in Model List

In [ ]:
def model_run(enable_write):
    for model in MODEL_LIST:
        name = model['name']
        path = model['path']
        scale = model['scale']

        sr=dnn_superres.DnnSuperResImpl_create()
        sr.readModel(path)
        sr.setModel(name.lower(),scale)

        for i in os.listdir(image_path):
            input_image=cv2.imread(os.path.join(image_path,i))
            input_image = cv2.resize(input_image, (int(input_image.shape[1]/scale),int(input_image.shape[0]/scale)))
            processed_image = cv2.resize(input_image, (int(input_image.shape[1]*scale),int(input_image.shape[0]*scale)))

            output_dir = os.path.join(output_path, f'x{scale}')
            output_file_path = os.path.join(output_dir, f'{name}_x{scale}_{i}')

            begin=time.time()
            result_image=sr.upsample(input_image)
            interval=time.time()-begin
            print(scale)
            print(name)
            print(f'Time is : {interval}')
            print(f'PSNR is : {calculate_psnr(processed_image, result_image)}')
            print(f'MSE is : {calculate_mse(processed_image, result_image)}')
            print(f'SSIM is : {calculate_ssim(processed_image, result_image)}')

            print('')
            if enable_write==True:
                save_image(output_file_path, result_image)

model_run(True)